# Web Scraper para busca do twitter

Uma ferramenta para coletar tweets da busca por termos do twitter. São os tweets de toda a rede(incluindo usuários que seu perfil não segue) marcados pela palavra-chave ou hashtag selecionada. Você pode incluir mais de um termo numa busca, mas tenha cuidado, pois sua amostra muito provavelmente será reduzida.

Este notebook é um guia! Você vai precisar rodar localmente o código, mas não pule essa etapa, pois é muito importante entender como funciona a ferramenta e como manipular os dados que você coletar.

Para obter acesso ao código completo clone o repositório: git@github.com:edumangabeira/crawlersTwitter.git

# Configurando o Ambiente
Pré-requisitos básicos:

1. Python3.x, pip3, tweepy, pymongo e mongoDB instalados.

# Conseguindo acesso aos dados da API

O Twitter se tornou mais restrito quanto a quem pode ter uma chave de acesso à sua API. É necessário pedir uma conta de desenvolvedor na página [apps.twitter.com] e justificar o seu uso.

Entre na sua conta do Twitter e em seguida crie uma aplicação na página [https://developer.twitter.com/en/apps]. O nome e a descrição não importam muito, faça como quiser. Para preencher o campo 'website URL' use qualquer link, até um que não seja registrado. 

Infelizmente a burocracia não acabou e você precisa descrever qual é seu objetivo fazendo esse app, a boa notícia é o mínimo ser de apenas 100 caracteres.

Ao final do processo, você terá à disposição algumas credenciais que iremos usar adiante.


## Exemplo de credenciais

#### É preciso mudar o trecho do código com as suas chaves

Consumer_key = uQl2J9OpHkqXcBKdrNjU09UB0

Consumer_secret = FdGsRUWESM3wrQVF6nDrOgsgi99CpwOvylA3yBt1m6EeJn1btg

Acess_token = 772557411177623882-GtuCaR7jApm6BGUVsnjyGXQIiHfKjiy

Acess_token_secret = d5n1VTrHIULSHSTF9VueoBOUZIlzsN7nFWkZkn0Ap1dPE2


### Atenção!

As credenciais acima são apenas ilustrativas. Quando você gerar suas credenciais, JAMAIS compartilhe elas em lugar algum.


# Bibliotecas


## Tweepy
Usaremos o Tweepy para estabelecer a comunicação com a API do Twitter, os métodos utilizados podem ser bem simples, mas aqui queremos poder extrair o máximo da biblioteca. o Tweepy vai lidar com a autenticação pelo protocolo OAuth requerida pelo Twitter e também irá cuidar das requisições

## Pymongo

Essa biblioteca irá nos ajudar a salvar os tweets coletados num banco de dados MongoDB.

## JSON

A API do twitter fornece os tweets no formato JSON. Caso encontre alguma dificuldade em manipular ou entender o formato, confira esse artigo: https://pythonhelp.wordpress.com/2013/03/21/acessando-conteudo-via-apis-web-baseadas-em-json/


# Acessando o mongoDB

Agora vai uma explicação rápida sobre o banco mongo, você pode pular essa parte por enquanto, pois só vai poder testar depois que todo o código for executado e os dados obtidos.

1) Para iniciar no seu terminal, basta digitar 'mongo'.

2) Primeiro é preciso ver quais bancos estão sendo usados na sua máquina, o comando a ser usado é 'show dbs'.

3) Para usar um banco: 'use {insira aqui um banco, sem chaves}'

4) Agora temos que selecionar uma coleção do nosso banco, para ver as coleções: 'show collections'.

5) Existem diferentes formas de acessar os dados, vou citar algumas.

5.1) 'db.{sua collection}.find()' mostra alguns tweets, é útil para verificar rapidamente erros em parâmetros. Também é possível filtrar a procura, para saber com mais detalhes veja em: https://docs.mongodb.com/manual/reference/method/db.collection.find/

5.2) 'db.{sua collection}.findOne()' mostra apenas um tweet e também pode filtrar a busca(recomendado nesse caso).

Se você deseja apenas um arquivo com a base de dados, volte para o seu terminal(crtl+C interrompe o processo mongo) e digite o seguinte comando:

##### Para obter .json (recomendado)
mongoexport --db {seu_banco} --collection {sua_coleção} --out {diretorio/nome_do_seu_arquivo}.json

##### Para obter .csv 
mongoexport --db {seu_banco} --collection {sua_coleção} --type=csv --out {diretorio/nome_do_seu_arquivo}.csv

Existem opções mais específicas não citadas aqui, como obter apenas alguns campos da tabela. Aqui está a documentação do MongoDB, caso queria usar mais alguma função não apresentada aqui. https://docs.mongodb.com/manual/tutorial/getting-started/

In [ ]:
# -*- coding: utf-8 -*-
import tweepy
import pymongo
import json
import sys
import time

# StdOutListener() e TimerBackOffToStream()

As duas classes foram feitas para garantir a conexão à API do twitter e configurar as escolhas de coleta(arquivo de log, banco de dados e coleção). Não se preocupe em configurar nada nelas, os métodos são apenas para garantir que os dados sejam salvos corretamente no banco e nossas credenciais não sejam bloqueadas pela API por excesso de tentativas de acesso.

## StdOutListener

Métodos para configurar arquivo de log, banco de dados e coleção do banco. As suas escolhas personalizadas devem ser alteradas na função main.

In [ ]:
class StdOutListener(tweepy.StreamListener):

    def __init__(self):
        self.logFile = ""
        self.collection = ""
        self.backOff = None

    def setLogFile(self, arq):
        self.logFile = arq

    def closeLogFile(self):
        if ((self.logFile is not None) and (self.logFile != "")):
            self.logFile.close()

    def setCollection(self, collection):
        self.collection = collection

    def setTimerBackOffToStream(self, backOff):
        self.backOff = backOff

    def on_data(self, data):
        if(self.collection == ""):
            print("Error: use setCollection para definir uma colecao no mongo")
            return

        data = json.loads(data)
        self.collection.insert(data)
        self.backOff.reiniciarContadorTentativas()

    def on_error(self, status):
        self.logFile.write("\n\nerror:" + str(status))
        self.backOff.timeReconexao(status)

## TimerBackOffToStream
Marca o tempo de espera até fazer uma requisição após uma tentativa.

In [ ]:
class TimerBackOffToStream(object):

    def __init__(self):
        self.tentativas = 1  # fator de multiplicacao

    def timeReconexao(self, HTTPerror=0):
        '''
            Avoiding to block the user's IP or credentials.
        '''
        if (HTTPerror == 420):
            time.sleep(60 * self.tentativas)
        elif (self.tentativas == 1):
            time.sleep(5)
        else:
            time.sleep(5 * self.tentativas * 2)
        self.tentativas += 1

    def setTentativas(self, incrementar):
        self.tentativas += incrementar

    def reiniciarContadorTentativas(self):
        self.tentativas = 1

# Executando a Main

Atenção! Não esqueça de preencher com as suas credenciais os campos:
1. Consumer_key = ""
2. Consumer_secret = ""
3. Access_token = ""
4. Access_token_secret = ""



Você também precisa ficar atento e mudar algumas configurações a cada coleta *distinta* que você fizer. As linhas de código se referem ao banco de dados usado e as coleções dele.

    Exemplo:
1. db = mongo['Hashtags_Bolsonaro'] # banco mongo
2. collection = db['BolsonaroInimigoDaEducação'] # uma única coleção

Lembre-se que você pode ter diversas coleções no mesmo banco.



# Linhas a alterar

##### arquivo de log

A sugestão é chamar o seu arquivo pelo mesmo nome da coleção no banco.

1. arqLog = open("logSeuArquivoDeLog.txt", "a")

##### banco de dados

Temos como exemplo um cenário em que se deseja coletar tweets que se relacionem com o atual presidente, Jair Bolsonaro. Esse banco vai guardar tweets com hashtags sobre o presidente.

2. db = mongo['Hashtags_Bolsonaro']

##### coleção do seu banco

Essa é uma coleção que guarda vários tweets com a mesma hashtag. Para hashtags distintas o melhor é criar uma nova coleção no mesmo banco.

3. collection = db['BolsonaroInimigoDaEducação']

##### termo a ser buscado

4. termos = ['#BolsonaroInimigoDaEducação']

A busca não precisa ser somente por hashtags e você pode escolher mais de um termo. Ex: ['eleição', 'Candidato','#2018']

In [1]:
if __name__ == '__main__':

    Consumer_key = ""
    Consumer_secret = ""
    Access_token = ""
    Access_token_secret = ""

    backOff = TimerBackOffToStream()
    arqLog = open("logSeuArquivoDeLog.txt", "a")

    while (True):
        try:
            listen = StdOutListener()

            mongo = pymongo.MongoClient()
            db = mongo['Hashtags_Bolsonaro']
            collection = db['BolsonaroInimigoDaEducação']

            listen.setCollection(collection)
            listen.setLogFile(arqLog)
            listen.setTimerBackOffToStream(backOff)

            auth = tweepy.OAuthHandler(Consumer_key, Consumer_secret)
            auth.set_access_token(Access_token, Access_token_secret)
            stream = tweepy.Stream(auth, listen)

            termos = ['#BolsonaroInimigoDaEducação']

            stream.filter(track=termos)

        except:
            print("except")
            arqLog.write("\nError: Exception:" + str(sys.exc_info()[0]))
            backOff.timeReconexao()

NameError: name 'TimerBackOffToStream' is not defined

# Rodando localmente

Para usar seu scraper baste digitar: python twitterStream_tweepy.py

Você pode alterar o nome do arquivo se tiver a intenção de criar um pra cada coleta, ex: twitterStream_tweepyImpeachmentBolso.py

# Carregando o dataset

A partir deste ponto você pode seguir como quiser, fica aqui apenas uma sugestão de como carregar o dataset.

Se você é minimamente familiar com a linguagem R, abra o Rstudio e digite no terminal dele os comandos abaixo.

> install.packages("jsonlite")

> library(jsonlite)

> nome_do_arquivo <- '/SeuLocalAqui/seuArquivo.json'

> dataset <-jsonlite::stream_in(textConnection(readLines(nome_do_arquivo, n=1000)),verbose=F)

Onde na última linha(dataset) o argumento "n=1000" do método readLines() se refere ao número máximo de linhas que você deseja visualizar, o que pode ser útil caso o dataset seja muito pesado e você não pretenda carregar tudo de uma vez.